In [1]:
#Standard Imports
import os
import sys
import pandas as pd
# from collections import defaultdict
import timeit
from docplex.cp.model import CpoModel

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.cp_model import CPModel
from src.music_functions import *

In [3]:
# Importing Musical Corpus
musical_work_df = pd.read_csv("../data/sample_input.csv")
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [4]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

In [5]:
# Choosing Musical Input
music = musical_corpus[0]
print(music.title, music.key, music.tonality, 
      music.first_on_beat,music.melody, music.reference_note)

Ach bleib' bei unsm Herr Jesu Christ 9 major 2 [37, 37, 40, 37, 33, 35, 37, 38, 37, 35, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33, 37, 35, 37, 33, 30, 32, 33, 35, 37, 33, 33, 33, 35, 37, 37, 35, 35, 33, 33, 33] 24


In [6]:
# Importing Chord Vocabulary
if music.tonality == "major":
    chord_df = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
else:
    chord_df = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab = []
for name, note_intervals in chord_df.itertuples():
    chord_vocab.append(Chord(name, set(int(x) for x in note_intervals.split(','))))

In [7]:
# Defining dictionary of weights for each soft constraint options:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
print(soft_constraint_w_weights)

{'chord progression': 2.0, 'chord repetition': 1.0, 'chord bass repetition': 10.0, 'leap resolution': 1.0, 'melodic movement': 1.0, 'note repetition': 1.0, 'parallel movement': 1.0, 'voice overlap': 1.0, 'adjacent bar chords': 1.0, 'chord spacing': 1.0, 'distinct notes': 1.0, 'voice crossing': 1.0, 'voice range': 1.0}


In [8]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing']
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords'] else False for x in hard_constraint_options}
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'voice crossing', 'voice range']

In [9]:
# Defining penalties for chord progression
if music.tonality == "major":
    penalties_chord_progression = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
else:
    penalties_chord_progression = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)

penalties_chord_progression = dict(penalties_chord_progression.stack())

In [10]:
# Model
cp_model = CPModel("test", musical_corpus[0], chord_vocab, penalties_chord_progression,
                   hard_constraints,
                   soft_constraint_w_weights)

In [11]:
solution = cp_model.solve()

(43680,)
-------------------------------------------------------------------------------
Model constraints: 122985, variables: integer: 859, interval: 0, sequence: 0
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 5.59 sec
-------------------------------------------------------------------------------
Objective values: (43680,), bounds: (43680,), gaps: (0,)
Chords_0=0
Chords_1=0
Chords_2=0
Chords_3=0
Chords_4=0
Chords_5=1
Chords_6=0
Chords_7=1
Chords_8=0
Chords_9=1
Chords_10=0
Chords_11=0
Chords_12=1
Chords_13=0
Chords_14=0
Chords_15=1
Chords_16=1
Chords_17=0
Chords_18=0
Chords_19=0
Chords_20=0
Chords_21=1
Chords_22=0
Chords_23=0
Chords_24=1
Chords_25=4
Chords_26=0
Chords_27=1
Chords_28=0
Chords_29=0
Chords_30=0
Chords_31=0
Chords_32=1
Chords_33=0
Chords_34=0
Chords_35=1
Chords_36=1
Chords_37=0
Chords_38=0
Chords_39=0
Notes_0=37
Notes_1=37
Notes_2=40
Notes_3=37
Notes_4=33
Notes_5=35
Notes_6=37
Notes_7=38
Notes_8=37
Notes_9=35
Notes_

In [12]:
solution_vars = cp_model.get_solution()

In [13]:
cp_model.export_midi()